In [91]:
import requests
import json

def rest_api_parameters(in_args, prefix='', out_dict=None):
    if out_dict==None:
        out_dict = {}
    if not type(in_args) in (list,dict):
        out_dict[prefix] = in_args
        return out_dict
    if prefix == '':
        prefix = prefix + '{0}'
    else:
        prefix = prefix + '[{0}]'
    if type(in_args)==list:
        for idx, item in enumerate(in_args):
            rest_api_parameters(item, prefix.format(idx), out_dict)
    elif type(in_args)==dict:
        for key, item in in_args.items():
            rest_api_parameters(item, prefix.format(key), out_dict)
    return out_dict

class MoodleCourseCreator:
    def __init__(self, domain, token):
        """
        Initialize with Moodle domain and Web Services token
        """
        self.domain = domain.rstrip('/')
        self.token = token
        self.base_url = f"{self.domain}/webservice/rest/server.php"

    def call_moodle_api(self, wsfunction, params=None):
        """Make a request to the Moodle API"""
        if params is None:
            params = {}
            
        parameters = rest_api_parameters(params)
        parameters.update({
            'wstoken': self.token,
            'moodlewsrestformat': 'json',
            'wsfunction': wsfunction
        })
        
        response = requests.post(self.base_url, data=parameters)
        print(f"API Call URL: {response.url}")
        
        if response.status_code != 200:
            raise Exception(f"API call failed with status code: {response.status_code}")
            
        try:
            result = response.json()
            if isinstance(result, dict) and 'exception' in result:
                raise Exception(f"Moodle API error: {result['message']}\nDebug info: {result.get('debuginfo', '')}")
            return result
        except json.JSONDecodeError as e:
            print(f"Raw response: {response.text}")
            raise Exception(f"Failed to decode JSON response: {str(e)}")

    def create_course(self, full_name, short_name, category_id=1, num_sections=0):
        """Create a new course with specified number of sections"""
        params = {
            'courses': [{
                'fullname': full_name,
                'shortname': short_name,
                'categoryid': category_id,
                'format': 'topics',
                'numsections': num_sections
            }]
        }
        
        result = self.call_moodle_api('core_course_create_courses', params)
        if isinstance(result, list) and result:
            return result[0].get('id')
        return None

    def get_course_sections(self, course_id):
        """Get the sections of a course"""
        return self.call_moodle_api('local_wsmanagesections_get_sections', {'courseid': course_id})

    def add_page_module(self, course_id, section, title, content):
        """Add a page module to a section"""
        params = {
            'courseid': course_id,
            'sectionnumber': section,
            'contentname': title,
            'contenttext': content,
            'contenttype': 'page'
        }
        return self.call_moodle_api('core_course_create_module', params)


# Example usage
MOODLE_URL = "http://localhost:8081/"
TOKEN = "3e532616b29a4a6e322f7c05ee407c79"

creator = MoodleCourseCreator(MOODLE_URL, TOKEN)

try:
    # Content sections with HTML content
    sections_data = [
        {
            "name": "Introduction to Python",
            "content": """
            <h2>Welcome to Python Programming</h2>
            <p>In this section, you will learn:</p>
            <ul>
                <li>What is Python?</li>
                <li>Setting up your development environment</li>
                <li>Running your first Python program</li>
            </ul>
            """
        },
        {
            "name": "Variables and Data Types",
            "content": """
            <h2>Python Variables and Data Types</h2>
            <p>Key concepts covered:</p>
            <ul>
                <li>Numbers (int, float)</li>
                <li>Strings and string operations</li>
                <li>Lists and tuples</li>
                <li>Dictionaries</li>
            </ul>
            """
        },
        {
            "name": "Control Structures",
            "content": """
            <h2>Control Flow in Python</h2>
            <p>Learn about:</p>
            <ul>
                <li>If-else statements</li>
                <li>For loops</li>
                <li>While loops</li>
                <li>Break and continue</li>
            </ul>
            """
        }
    ]
    
    # Create the course
    course_id = creator.create_course(
        full_name="mycourse-01",
        short_name="mycourse-01",
        category_id=1,
        num_sections=len(sections_data)
    )
    print(f"Created course with ID: {course_id}")

    if course_id:
        # Add content to each section
        for index, section in enumerate(sections_data, start=1):
            try:
                print(f"\nAdding content to section {index}")
                result = creator.add_page_module(
                    course_id=course_id,
                    section=index,
                    title=section["name"],
                    content=section["content"]
                )
                print(f"Added page module: {result}")
            except Exception as e:
                print(f"Error adding content to section {index}: {str(e)}")

        print("\nFinal course structure:")
        final_sections = creator.get_course_sections(course_id)
        print(json.dumps(final_sections, indent=2))
        
        print(f"\nCourse setup completed. Course ID: {course_id}")
        print(f"You can view the course at: {MOODLE_URL}/course/view.php?id={course_id}")
    else:
        print("Failed to create course")

except Exception as e:
    print(f"Error: {str(e)}")
    print("\nPlease ensure these Web Services functions are enabled:")
    print("- core_course_create_courses")
    print("- core_course_get_contents")
    print("- core_course_create_module")

API Call URL: http://localhost:8081/webservice/rest/server.php
Created course with ID: 21

Adding content to section 1
API Call URL: http://localhost:8081/webservice/rest/server.php
Error adding content to section 1: Moodle API error: Can't find data record in database table external_functions.
Debug info: SELECT * FROM {external_functions} WHERE name = ?
[array (
  0 => 'core_course_create_module',
)]

Adding content to section 2
API Call URL: http://localhost:8081/webservice/rest/server.php
Error adding content to section 2: Moodle API error: Can't find data record in database table external_functions.
Debug info: SELECT * FROM {external_functions} WHERE name = ?
[array (
  0 => 'core_course_create_module',
)]

Adding content to section 3
API Call URL: http://localhost:8081/webservice/rest/server.php
Error adding content to section 3: Moodle API error: Can't find data record in database table external_functions.
Debug info: SELECT * FROM {external_functions} WHERE name = ?
[array (
 

In [98]:
sections = creator.call_moodle_api('local_wsmanagesections_get_sections', {"courseid": course_id})
sections_data = [
    {'type': 'num', 'section': 0, 'summary': '<h2>Welcome to Python Programming</h2> <p>In this section, you will learn:</p> <ul> <li>What is Python?</li> <li>Setting up your development environment</li> <li>Running your first Python program</li> </ul>', 'name': 'Four§' , 'summaryformat': 1, 'visible': 1 , 'highlight': 0, 'sectionformatoptions': [{'name': 'level', 'value': '1'}]},\
    {'type': 'num', 'section': 1, 'summary': '<h2>Welcome to Python Programming</h2> <p>In this section, you will learn:</p> <ul> <li>What is Python?</li> <li>Setting up your development environment</li> <li>Running your first Python program</li> </ul>', 'name': 'Four§' , 'summaryformat': 1, 'visible': 1 , 'highlight': 0, 'sectionformatoptions': [{'name': 'level', 'value': '1'}]},\
    {'type': 'num', 'section': 2, 'summary': '<h2>Welcome to Python Programming</h2> <p>In this section, you will learn:</p> <ul> <li>What is Python?</li> <li>Setting up your development environment</li> <li>Running your first Python program</li> </ul>', 'name': 'Four§' , 'summaryformat': 1, 'visible': 1 , 'highlight': 0, 'sectionformatoptions': [{'name': 'level', 'value': '1'}]},\
    {'type': 'num', 'section': 3, 'summary': '<h2>Welcome to Python Programming</h2> <p>In this section, you will learn:</p> <ul> <li>What is Python?</li> <li>Setting up your development environment</li> <li>Running your first Python program</li> </ul>', 'name': 'Four§' , 'summaryformat': 1, 'visible': 1 , 'highlight': 0, 'sectionformatoptions': [{'name': 'level', 'value': '1'}]},\
]

result = creator.call_moodle_api('local_wsmanagesections_update_sections', {"courseid": course_id, "sections": sections_data})

API Call URL: http://localhost:8081/webservice/rest/server.php
API Call URL: http://localhost:8081/webservice/rest/server.php


In [77]:
from requests import get, post

# Module variables to connect to moodle api:
## Insert token and URL for your site here. 
## Mind that the endpoint can start with "/moodle" depending on your installation.
URL = "http://localhost:8081"
KEY = "3e532616b29a4a6e322f7c05ee407c79"
ENDPOINT="/webservice/rest/server.php"

def rest_api_parameters(in_args, prefix='', out_dict=None):
    """Transform dictionary/array structure to a flat dictionary, with key names
    defining the structure.
    Example usage:
    >>> rest_api_parameters({'courses':[{'id':1,'name': 'course1'}]})
    {'courses[0][id]':1,
     'courses[0][name]':'course1'}
    """
    if out_dict==None:
        out_dict = {}
    if not type(in_args) in (list,dict):
        out_dict[prefix] = in_args
        return out_dict
    if prefix == '':
        prefix = prefix + '{0}'
    else:
        prefix = prefix + '[{0}]'
    if type(in_args)==list:
        for idx, item in enumerate(in_args):
            rest_api_parameters(item, prefix.format(idx), out_dict)
    elif type(in_args)==dict:
        for key, item in in_args.items():
            rest_api_parameters(item, prefix.format(key), out_dict)
    return out_dict

def call(fname, **kwargs):
    """Calls moodle API function with function name fname and keyword arguments.
    Example:
    >>> call_mdl_function('core_course_update_courses',
                           courses = [{'id': 1, 'fullname': 'My favorite course'}])
    """
    parameters = rest_api_parameters(kwargs)
    parameters.update({"wstoken": KEY, 'moodlewsrestformat': 'json', "wsfunction": fname})
    #print(parameters)
    response = post(URL+ENDPOINT, data=parameters).json()
    if type(response) == dict and response.get('exception'):
        raise SystemError("Error calling Moodle API\n", response)
    return response

################################################
# Rest-Api classes
################################################

class LocalUpdateSections(object):
    """Updates sectionnames. Requires: courseid and an array with sectionnumbers and sectionnames"""
    def __init__(self, cid, sectionsdata):
        self.updatesections = call('local_wsmanagesections_update_sections', courseid = cid, sections = sectionsdata)
        
################################################
# Example
################################################

# Update sections. Example for onetopic format.
courseid = course_id # Exchange with valid id.
data = [
    {'type': 'num', 'section': 0, 'name': 'Four§', 'summary': '<p>section FOUR</p>', 'summaryformat': 1, 'visible': 1 , 'highlight': 0, 'sectionformatoptions': [{'name': 'level', 'value': '1'}]},\
    {'type': 'num', 'section': 2, 'name': '3', 'summary': '<p>section THREE</p>', 'summaryformat': 1, 'visible': 0 , 'highlight': 0, 'sectionformatoptions': [{'name': 'level', 'value': '0'}]}
]
sec = LocalUpdateSections(courseid, data)
print(sec.updatesections)
print(course_id)

{'warnings': []}
20
